# M2177.003100 Deep Learning <br> Final Proejct: Text to Image Synthesis (Tensorflow)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PPT file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the training process **</font> so that TAs can grade both your code and results.  
**The TA will set a config file as 'eval_birds.yml' when evaluating the code using 'hidden test dataset'. Thus, please make sure that your code can generate proper data to measure inception score and R-precision of 'hidden test dataset'.**

## 1. Load datasets
The Birds dataset will be downloaded automatically if it is not located in the *data* directory. <br>

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os, nltk

from miscc.config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz
import numpy as np
import scipy
import time
import random

from utils.data_utils import CUBDataset
from utils.loss import cosine_similarity

from utils.data_utils import *

#################################################
# DO NOT CHANGE 
from utils.model import CNN_ENCODER, RNN_ENCODER, GENERATOR, DISCRIMINATOR
#################################################

%matplotlib inline

In [3]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/train_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = '3' #cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = 'sample/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

Using config:
{'BATCH_SIZE': 32,
 'CHECKPOINT_DIR': './checkpoint',
 'CHECKPOINT_NAME': 'model.ckpt',
 'CNN': {'EMBEDDING_DIM': 0, 'H_DIM': 0},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': False,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': False,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 0,
         'DF_DIM': 0,
         'EMBEDDING_DIM': 0,
         'GF_DIM': 0,
         'R_NUM': 0,
         'Z_DIM': 512},
 'GPU_ID': '0',
 'IMAGE_SIZE': 256,
 'NUM_BATCH_FOR_TEST': 0,
 'RANDOM_SEED': 0,
 'RNN': {'EMBEDDING_DIM': 0,
         'H_DIM': 0,
         'TYPE': '',
         'VOCAB_SIZE': 0,
         'WORD_EMBEDDING_DIM': 0},
 'R_PRECISION_DIR': './evaluation',
 'R_PRECISION_FILE': 'r_precision.npz',
 'R_PRECISION_FILE_HIDDEN': 'r_precision_hidden.npz',
 'TEST': {'B_EXAMPLE': False,
          'GENERATED_HIDDEN_TEST_IMAGES': './evaluation/generated_images_hidden',
          'GENERATED_TEST_IMAGES'

/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf/miscc/config.py:121: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [4]:
train_dataset = CUBDataset(cfg.DATA_DIR, split='train')
test_dataset = CUBDataset(cfg.DATA_DIR, split='test')

print(f'\ntrain data directory:\n{train_dataset.split_dir}')
print(f'test data directory:\n{test_dataset.split_dir}\n')

print(f'# of train filenames:{train_dataset.filenames.shape}')
print(f'# of test filenames:{test_dataset.filenames.shape}\n')

print(f'example of filename of train image:{train_dataset.filenames[0]}')
print(f'example of filename of valid image:{test_dataset.filenames[0]}\n')

print(f'example of caption and its ids:\n{train_dataset.captions[0]}\n{train_dataset.captions_ids[0]}\n')
print(f'example of caption and its ids:\n{test_dataset.captions[0]}\n{test_dataset.captions_ids[0]}\n')

print(f'# of train captions:{np.asarray(train_dataset.captions).shape}')
print(f'# of test captions:{np.asarray(test_dataset.captions).shape}\n')

print(f'# of train caption ids:{np.asarray(train_dataset.captions_ids).shape}')
print(f'# of test caption ids:{np.asarray(test_dataset.captions_ids).shape}\n')

print(f'# of train images:{train_dataset.images.shape}')
print(f'# of test images:{test_dataset.images.shape}\n')

self.current_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf

self.data_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf/data/birds/CUB_200_2011

Dataset already exists
self.image_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle
self.current_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf

self.data_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-project-deep-learning-19-tf/data/birds/CUB_200_2011

Dataset already exists
self.image_dir:
/home/duclv/homework/deep-learning-2019/final_project/final-p

## 2. Define models and go to train/evaluate

In [5]:
###======================== DEFIINE VARIABLES ===================================###
lr = 0.0002
lr_decay = 0.5      
decay_every = 100  
beta1 = 0.5
batch_size = cfg.BATCH_SIZE
image_size = cfg.IMAGE_SIZE
z_dim = cfg.GAN.Z_DIM

tf.reset_default_graph()
from importlib import reload
import utils.model as model
model = reload(model)
RNN_ENCODER = model.RNN_ENCODER
GENERATOR = model.GENERATOR
DISCRIMINATOR = model.DISCRIMINATOR
CNN_ENCODER = model.CNN_ENCODER

In [6]:
###======================== DEFIINE PLACEHOLDER ===================================###
t_real_image = tf.placeholder('float32', [cfg.BATCH_SIZE, cfg.IMAGE_SIZE, cfg.IMAGE_SIZE, 3], name = 'real_image')
t_real_caption = tf.placeholder(dtype=tf.int64, shape=[cfg.BATCH_SIZE , None], name='real_caption_input')
t_wrong_image = tf.placeholder('float32', [cfg.BATCH_SIZE ,cfg.IMAGE_SIZE, cfg.IMAGE_SIZE, 3], name = 'wrong_image')
t_wrong_caption = tf.placeholder(dtype=tf.int64, shape=[cfg.BATCH_SIZE , None], name='wrong_caption_input')
t_z = tf.placeholder(tf.float32, [cfg.BATCH_SIZE , cfg.GAN.Z_DIM], name='z_noise')

In [7]:
rnn_encoder = RNN_ENCODER(t_real_caption, cfg.BATCH_SIZE, is_training=False, reuse=False)
generator = GENERATOR(t_z, rnn_encoder.outputs, is_training=False, reuse=False)
discriminator = DISCRIMINATOR(generator.outputs, rnn_encoder.outputs, is_training=False, reuse=False)
cnn_encoder = CNN_ENCODER(t_real_image, is_training=False, reuse=False)

In [8]:
### Define image and text mapping
net_cnn = CNN_ENCODER(t_real_image, is_training=True, reuse=True)
x = net_cnn.outputs
v = RNN_ENCODER(t_real_caption, cfg.BATCH_SIZE, is_training=True, reuse=True).outputs
x_w = CNN_ENCODER(t_wrong_image, is_training=True, reuse=True).outputs
v_w = RNN_ENCODER(t_wrong_caption, cfg.BATCH_SIZE, is_training=True, reuse=True).outputs

alpha = 0.2 # margin alpha
rnn_loss = tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(x, v) + cosine_similarity(x, v_w))) + \
            tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(x, v) + cosine_similarity(x_w, v)))

In [9]:
### Define generative model
net_rnn = RNN_ENCODER(t_real_caption, cfg.BATCH_SIZE, is_training=False, reuse=True)
net_fake_image = GENERATOR(t_z, net_rnn.outputs, is_training=True, reuse=True)

net_disc_fake = DISCRIMINATOR(net_fake_image.outputs, net_rnn.outputs, is_training=True, reuse=True)
disc_fake_logits = net_disc_fake.logits

net_disc_real = DISCRIMINATOR(t_real_image, net_rnn.outputs, is_training=True, reuse=True)
disc_real_logits = net_disc_real.logits

net_disc_mismatch = DISCRIMINATOR(t_real_image, 
                            RNN_ENCODER(t_wrong_caption, cfg.BATCH_SIZE, is_training=False, reuse=True).outputs,
                            is_training=True, reuse=True)
disc_mismatch_logits = net_disc_mismatch.logits

d_loss1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_real_logits,     labels=tf.ones_like(disc_real_logits),      name='d1'))
d_loss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_mismatch_logits, labels=tf.zeros_like(disc_mismatch_logits), name='d2'))
d_loss3 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_logits,     labels=tf.zeros_like(disc_fake_logits),     name='d3'))
d_loss = d_loss1 + (d_loss2 + d_loss3) * 0.5

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_logits, labels=tf.ones_like(disc_fake_logits), name='g'))

In [10]:
### Define optimzers
rnn_vars = [var for var in tf.trainable_variables() if 'rnnencoder' in var.name]
g_vars = [var for var in tf.trainable_variables() if 'generator' in var.name]
d_vars = [var for var in tf.trainable_variables() if 'discriminator' in var.name]
cnn_vars = [var for var in tf.trainable_variables() if 'cnnencoder' in var.name]

update_ops_D = [var for var in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'discriminator' in var.name]
update_ops_G = [var for var in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'generator' in var.name]
update_ops_CNN = [var for var in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'cnnencoder' in var.name]

'''print('----------Update_ops_D--------')
for var in update_ops_D:
    print(var.name)
print('----------Update_ops_G--------')
for var in update_ops_G:
    print(var.name)
print('----------Update_ops_CNN--------')
for var in update_ops_CNN:
    print(var.name)'''

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr, trainable=False)

with tf.control_dependencies(update_ops_D):
    d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

with tf.control_dependencies(update_ops_G):
    g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)

with tf.control_dependencies(update_ops_CNN):
    grads, _ = tf.clip_by_global_norm(tf.gradients(rnn_loss, rnn_vars + cnn_vars), 10)
    optimizer = tf.train.AdamOptimizer(lr_v, beta1=beta1)
    rnn_optim = optimizer.apply_gradients(zip(grads, rnn_vars + cnn_vars))

In [11]:
import threading
import scipy.ndimage as ndi
from skimage import transform
from skimage import exposure
import skimage

# Data Augmentation reference: https://github.com/tensorlayer/tensorlayer/tree/master/tensorlayer
def threading_data(data=None, fn=None, **kwargs):
    def apply_fn(results, i, data, kwargs):
        results[i] = data #fn(data, **kwargs)

    ## start multi-threaded reading.
    results = [None] * len(data) ## preallocate result list
    threads = []
    for i in range(len(data)):
        t = threading.Thread(
                        name='threading_and_return',
                        target=apply_fn,
                        args=(results, i, data[i], kwargs)
                        )
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

    return np.asarray(results)

def apply_transform(x, transform_matrix, channel_index=2, fill_mode='nearest', cval=0., order=1):
    x = np.rollaxis(x, channel_index, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(x_channel, final_affine_matrix,
                      final_offset, order=order, mode=fill_mode, cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_index+1)
    return x

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

def rotation(x, rg=20, is_random=False, row_index=0, col_index=1, channel_index=2,
                    fill_mode='nearest', cval=0.):
    if is_random:
        theta = np.pi / 180 * np.random.uniform(-rg, rg)
    else:
        theta = np.pi /180 * rg
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])

    h, w = x.shape[row_index], x.shape[col_index]
    transform_matrix = transform_matrix_offset_center(rotation_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_index, fill_mode, cval)
    return x

def crop(x, wrg, hrg, is_random=False, row_index=0, col_index=1, channel_index=2):
    h, w = x.shape[row_index], x.shape[col_index]
    assert (h > hrg) and (w > wrg), "The size of cropping should smaller than the original image"
    if is_random:
        h_offset = int(np.random.uniform(0, h-hrg) -1)
        w_offset = int(np.random.uniform(0, w-wrg) -1)
        return x[h_offset: hrg+h_offset ,w_offset: wrg+w_offset]
    else:   # central crop
        h_offset = int(np.floor((h - hrg)/2.))
        w_offset = int(np.floor((w - wrg)/2.))
        h_end = h_offset + hrg
        w_end = w_offset + wrg
        return x[h_offset: h_end, w_offset: w_end]

def flip_axis(x, axis, is_random=False):
    if is_random:
        factor = np.random.uniform(-1, 1)
        if factor > 0:
            x = np.asarray(x).swapaxes(axis, 0)
            x = x[::-1, ...]
            x = x.swapaxes(0, axis)
            return x
        else:
            return x
    else:
        x = np.asarray(x).swapaxes(axis, 0)
        x = x[::-1, ...]
        x = x.swapaxes(0, axis)
        return x

def imresize(x, size=[100, 100], interp='bilinear', mode=None):
    if x.shape[-1] == 1:
        # greyscale
        x = scipy.misc.imresize(x[:,:,0], size, interp=interp, mode=mode)
        return x[:, :, np.newaxis]
    elif x.shape[-1] == 3:
        # rgb, bgr ..
        return scipy.misc.imresize(x, size, interp=interp, mode=mode)
    else:
        raise Exception("Unsupported channel %d" % x.shape[-1])

def prepro_img(x, mode=None):
    # rescale [0, 255] --> (-1, 1), random flip, crop, rotate

    if mode=='train':
        x = flip_axis(x, axis=1, is_random=True)
        x = rotation(x, rg=16, is_random=True, fill_mode='nearest')
        x = imresize(x, size=[image_size+35, image_size+35], interp='bilinear', mode=None)
        x = crop(x, wrg=image_size, hrg=image_size, is_random=True)
        x = x / (255. / 2.)
        x = x - 1.
        # x = x * 0.9999

    return x

def get_random_int(min=0, max=10, number=5):
    """Return a list of random integer by the given range and quantity.
    Examples
    ---------
    >>> r = get_random_int(min=0, max=10, number=5)
    ... [10, 2, 3, 3, 7]
    """
    return [random.randint(min,max) for p in range(0,number)]

In [ ]:
### Train
sess = tf.Session(config=tf.ConfigProto())
init = tf.global_variables_initializer()
sess.run(init)

#saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=100)
saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=5)
checkpoint_dir = cfg.CHECKPOINT_DIR
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
'''if ckpt and ckpt.model_checkpoint_path:
    loader = tf.train.Saver(var_list=tf.global_variables())
    load_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Restored model parameters from {}".format(ckpt_path))
else:
    print('no checkpoints find.')'''

n_epoch = cfg.TRAIN.MAX_EPOCH
n_images_train = len(train_dataset.images)
n_batch_epoch = int(n_images_train / batch_size)
train_captions = np.array(train_dataset.captions_ids)
train_images = np.array(train_dataset.images)
n_captions_train = len(train_captions)
n_captions_per_image = cfg.TEXT.CAPTIONS_PER_IMAGE

for epoch in range(cfg.TRAIN.MAX_EPOCH):
    #################################################
    # TODO: Implement text to image synthesis
    start_time = time.time()

    if epoch !=0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay ** (epoch // decay_every)
        sess.run(tf.assign(lr_v, lr * new_lr_decay))
        log = " ** new learning rate: %f" % (lr * new_lr_decay)
        print(log)

    elif epoch == 0:
        log = " ** init lr: %f  decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
        print(log)
    
    for step in range(n_batch_epoch):
        step_time = time.time()

        ## get matched text & image
        idexs = get_random_int(min=0, max=n_captions_train-1, number=batch_size)
        b_real_caption = train_captions[idexs]
        b_real_images = train_images[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]

        """ check for loading right images
        save_images(b_real_images, [ni, ni], 'train_samples/train_00.png')
        for caption in b_real_caption[:8]:
            print(IdList2sent(caption))
        exit()
        """

        ## get wrong caption & wrong image
        idexs = get_random_int(min=0, max=n_captions_train-1, number=batch_size)
        b_wrong_caption = train_captions[idexs]
        idexs2 = get_random_int(min=0, max=n_images_train-1, number=batch_size)
        b_wrong_images = train_images[idexs2]

        ## get noise
        b_z = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim)).astype(np.float32)

        b_real_images = threading_data(b_real_images, prepro_img, mode='train')   # [0, 255] --> [-1, 1] + augmentation
        b_wrong_images = threading_data(b_wrong_images, prepro_img, mode='train')

        ## update RNN
        if epoch < 80:
            errRNN, _ = sess.run([rnn_loss, rnn_optim], feed_dict={
                                            t_real_image : b_real_images,
                                            t_wrong_image : b_wrong_images,
                                            t_real_caption : b_real_caption,
                                            t_wrong_caption : b_wrong_caption})
        else:
            errRNN = 0

        ## updates D
        errD, _ = sess.run([d_loss, d_optim], feed_dict={
                        t_real_image : b_real_images,
                        t_wrong_caption : b_wrong_caption,
                        t_real_caption : b_real_caption,
                        t_z : b_z})
        ## updates G
        errG, _ = sess.run([g_loss, g_optim], feed_dict={
                        t_real_caption : b_real_caption,
                        t_z : b_z})

        print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_loss: %.8f, g_loss: %.8f, rnn_loss: %.8f" \
                    % (epoch, n_epoch, step, n_batch_epoch, time.time() - step_time, errD, errG, errRNN))

    if (epoch + 1) % 1 == 0:
        print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
        '''img_gen, rnn_out = sess.run([net_g.outputs, net_rnn.outputs], feed_dict={
                                    t_real_caption : sample_sentence,
                                    t_z : sample_seed})

        save_images(img_gen, [ni, ni], 'train_samples/train_{:02d}.png'.format(epoch))'''

    if (epoch != 0) and (epoch % 10) == 0:
        #################################################
        # save checkpoints
        checkpoint_path = os.path.join(cfg.CHECKPOINT_DIR, cfg.CHECKPOINT_NAME)
        saver.save(sess, checkpoint_path, global_step=epoch)
        print('The checkpoint has been created.')

 ** init lr: 0.000200  decay_every_epoch: 100, lr_decay: 0.500000
Epoch: [ 0/600] [   0/ 276] time: 55.2869s, d_loss: 4.16007280, g_loss: 0.07942918, rnn_loss: 0.39139336
Epoch: [ 0/600] [   1/ 276] time: 3.7368s, d_loss: 13.87337589, g_loss: 22.77612686, rnn_loss: 0.42251897
Epoch: [ 0/600] [   2/ 276] time: 3.7422s, d_loss: 4.09953928, g_loss: 16.37288857, rnn_loss: 0.35128343
Epoch: [ 0/600] [   3/ 276] time: 3.7483s, d_loss: 6.11808014, g_loss: 6.61497927, rnn_loss: 0.39881286
Epoch: [ 0/600] [   4/ 276] time: 3.7411s, d_loss: 5.01200771, g_loss: 30.65848160, rnn_loss: 0.41313750
Epoch: [ 0/600] [   5/ 276] time: 3.7104s, d_loss: 6.32387066, g_loss: 24.52893829, rnn_loss: 0.41684318
Epoch: [ 0/600] [   6/ 276] time: 3.7585s, d_loss: 3.65051436, g_loss: 15.54294491, rnn_loss: 0.36780831
Epoch: [ 0/600] [   7/ 276] time: 3.5741s, d_loss: 5.12466049, g_loss: 18.29709435, rnn_loss: 0.47264713
Epoch: [ 0/600] [   8/ 276] time: 3.5599s, d_loss: 4.25928974, g_loss: 13.77301407, rnn_loss: 

Epoch: [ 0/600] [  78/ 276] time: 3.5808s, d_loss: 2.80912209, g_loss: 5.63739300, rnn_loss: 0.35911918
Epoch: [ 0/600] [  79/ 276] time: 3.6412s, d_loss: 2.24622297, g_loss: 5.87053108, rnn_loss: 0.42442584
Epoch: [ 0/600] [  80/ 276] time: 3.6153s, d_loss: 2.89113855, g_loss: 4.98837280, rnn_loss: 0.33477572
Epoch: [ 0/600] [  81/ 276] time: 3.6137s, d_loss: 3.10320926, g_loss: 5.26163244, rnn_loss: 0.47184196
Epoch: [ 0/600] [  82/ 276] time: 3.5878s, d_loss: 1.97518945, g_loss: 2.08299780, rnn_loss: 0.41990560
Epoch: [ 0/600] [  83/ 276] time: 3.5959s, d_loss: 2.55480266, g_loss: 9.06345367, rnn_loss: 0.37490943
Epoch: [ 0/600] [  84/ 276] time: 3.5842s, d_loss: 2.37568498, g_loss: 4.30137825, rnn_loss: 0.38230443
Epoch: [ 0/600] [  85/ 276] time: 3.5800s, d_loss: 2.50229526, g_loss: 8.04148293, rnn_loss: 0.40785220
Epoch: [ 0/600] [  86/ 276] time: 3.5904s, d_loss: 2.36605024, g_loss: 4.94594145, rnn_loss: 0.38502944
Epoch: [ 0/600] [  87/ 276] time: 3.5803s, d_loss: 2.71006918, g

Epoch: [ 0/600] [ 157/ 276] time: 3.5796s, d_loss: 2.71002436, g_loss: 3.02970243, rnn_loss: 0.39578730
Epoch: [ 0/600] [ 158/ 276] time: 3.5779s, d_loss: 1.97470403, g_loss: 2.31319284, rnn_loss: 0.38005447
Epoch: [ 0/600] [ 159/ 276] time: 3.5835s, d_loss: 1.95334911, g_loss: 4.73461533, rnn_loss: 0.39319861
Epoch: [ 0/600] [ 160/ 276] time: 3.5895s, d_loss: 1.62461960, g_loss: 3.16406274, rnn_loss: 0.37456971
Epoch: [ 0/600] [ 161/ 276] time: 3.5781s, d_loss: 1.95011580, g_loss: 2.00341415, rnn_loss: 0.36516267
Epoch: [ 0/600] [ 162/ 276] time: 3.5830s, d_loss: 2.36315393, g_loss: 4.16792488, rnn_loss: 0.39587057
Epoch: [ 0/600] [ 163/ 276] time: 3.6112s, d_loss: 2.63088894, g_loss: 1.58857703, rnn_loss: 0.41746259
Epoch: [ 0/600] [ 164/ 276] time: 3.5793s, d_loss: 2.59066772, g_loss: 4.02055740, rnn_loss: 0.39489135
Epoch: [ 0/600] [ 165/ 276] time: 3.6149s, d_loss: 1.91531086, g_loss: 2.80771422, rnn_loss: 0.41245684
Epoch: [ 0/600] [ 166/ 276] time: 3.6016s, d_loss: 2.16514158, g

Epoch: [ 0/600] [ 236/ 276] time: 3.5863s, d_loss: 1.82462764, g_loss: 1.57256961, rnn_loss: 0.30304429
Epoch: [ 0/600] [ 237/ 276] time: 3.5959s, d_loss: 2.69437599, g_loss: 1.30602264, rnn_loss: 0.36016408
Epoch: [ 0/600] [ 238/ 276] time: 3.5811s, d_loss: 1.94722581, g_loss: 3.86980295, rnn_loss: 0.32490924
Epoch: [ 0/600] [ 239/ 276] time: 3.5934s, d_loss: 1.59851527, g_loss: 1.73156118, rnn_loss: 0.28559917
Epoch: [ 0/600] [ 240/ 276] time: 3.5757s, d_loss: 2.20682359, g_loss: 1.90149570, rnn_loss: 0.36417413
Epoch: [ 0/600] [ 241/ 276] time: 3.5565s, d_loss: 2.56060553, g_loss: 3.70670652, rnn_loss: 0.33964613
Epoch: [ 0/600] [ 242/ 276] time: 3.5537s, d_loss: 2.01462531, g_loss: 2.16558313, rnn_loss: 0.33073497
Epoch: [ 0/600] [ 243/ 276] time: 3.6083s, d_loss: 2.40315628, g_loss: 2.62518811, rnn_loss: 0.36386043
Epoch: [ 0/600] [ 244/ 276] time: 3.5890s, d_loss: 1.86878610, g_loss: 2.94165087, rnn_loss: 0.34604123
Epoch: [ 0/600] [ 245/ 276] time: 3.5805s, d_loss: 2.51315594, g

Epoch: [ 1/600] [  39/ 276] time: 3.6162s, d_loss: 1.96083903, g_loss: 0.96206719, rnn_loss: 0.34574950
Epoch: [ 1/600] [  40/ 276] time: 3.6022s, d_loss: 1.89862180, g_loss: 2.55492687, rnn_loss: 0.31253779
Epoch: [ 1/600] [  41/ 276] time: 3.5792s, d_loss: 1.67720819, g_loss: 1.87301707, rnn_loss: 0.33143771
Epoch: [ 1/600] [  42/ 276] time: 3.5698s, d_loss: 1.86976624, g_loss: 2.49510264, rnn_loss: 0.33956051
Epoch: [ 1/600] [  43/ 276] time: 3.5806s, d_loss: 2.04730392, g_loss: 2.55108929, rnn_loss: 0.28494644
Epoch: [ 1/600] [  44/ 276] time: 3.5829s, d_loss: 2.28556347, g_loss: 0.58043146, rnn_loss: 0.34361982
Epoch: [ 1/600] [  45/ 276] time: 3.5262s, d_loss: 2.26557136, g_loss: 3.02004743, rnn_loss: 0.27582192
Epoch: [ 1/600] [  46/ 276] time: 3.5800s, d_loss: 1.70706797, g_loss: 2.60828447, rnn_loss: 0.29503202
Epoch: [ 1/600] [  47/ 276] time: 3.6236s, d_loss: 1.71549439, g_loss: 0.70891058, rnn_loss: 0.28627145
Epoch: [ 1/600] [  48/ 276] time: 3.6963s, d_loss: 2.09692812, g

...

Epoch: [ 7/600] [ 121/ 276] time: 3.9690s, d_loss: 1.41818166, g_loss: 4.49922371, rnn_loss: 0.16928652
Epoch: [ 7/600] [ 122/ 276] time: 6.7241s, d_loss: 1.02809811, g_loss: 4.60924816, rnn_loss: 0.18840846
Epoch: [ 7/600] [ 123/ 276] time: 5.6619s, d_loss: 1.23654997, g_loss: 2.15287995, rnn_loss: 0.16358405
Epoch: [ 7/600] [ 124/ 276] time: 4.8720s, d_loss: 1.50852728, g_loss: 4.10394955, rnn_loss: 0.16917309
Epoch: [ 7/600] [ 125/ 276] time: 5.7716s, d_loss: 0.97308266, g_loss: 3.47506881, rnn_loss: 0.17038813
Epoch: [ 7/600] [ 126/ 276] time: 5.2480s, d_loss: 1.11090016, g_loss: 2.62948370, rnn_loss: 0.24677266
Epoch: [ 7/600] [ 127/ 276] time: 5.6449s, d_loss: 1.41278231, g_loss: 4.00973129, rnn_loss: 0.18896480
Epoch: [ 7/600] [ 128/ 276] time: 4.6128s, d_loss: 1.44031096, g_loss: 1.97261596, rnn_loss: 0.15756361
Epoch: [ 7/600] [ 129/ 276] time: 4.5841s, d_loss: 1.39546561, g_loss: 2.16090298, rnn_loss: 0.23637238
Epoch: [ 7/600] [ 130/ 276] time: 3.5221s, d_loss: 1.18122053, g

Epoch: [ 7/600] [ 200/ 276] time: 3.6973s, d_loss: 1.58006322, g_loss: 0.87952256, rnn_loss: 0.20055839
Epoch: [ 7/600] [ 201/ 276] time: 6.8798s, d_loss: 1.28012252, g_loss: 2.82862663, rnn_loss: 0.15765420
Epoch: [ 7/600] [ 202/ 276] time: 3.9320s, d_loss: 0.91271067, g_loss: 3.05593872, rnn_loss: 0.11913800
Epoch: [ 7/600] [ 203/ 276] time: 6.7265s, d_loss: 1.04675674, g_loss: 1.63355613, rnn_loss: 0.25645104
Epoch: [ 7/600] [ 204/ 276] time: 3.7000s, d_loss: 1.75182867, g_loss: 0.68622828, rnn_loss: 0.13564086
Epoch: [ 7/600] [ 205/ 276] time: 6.5717s, d_loss: 1.88415015, g_loss: 4.62045097, rnn_loss: 0.22009626
Epoch: [ 7/600] [ 206/ 276] time: 4.1569s, d_loss: 2.43136597, g_loss: 1.58299780, rnn_loss: 0.24775025
Epoch: [ 7/600] [ 207/ 276] time: 7.1378s, d_loss: 1.37673461, g_loss: 1.68105245, rnn_loss: 0.23869115
Epoch: [ 7/600] [ 208/ 276] time: 4.2408s, d_loss: 1.20255959, g_loss: 4.33334875, rnn_loss: 0.15312168
Epoch: [ 7/600] [ 209/ 276] time: 3.4947s, d_loss: 1.62693906, g

Epoch: [ 8/600] [   3/ 276] time: 5.3288s, d_loss: 0.99020088, g_loss: 2.82469034, rnn_loss: 0.21084729
Epoch: [ 8/600] [   4/ 276] time: 5.5536s, d_loss: 1.48921406, g_loss: 2.17799640, rnn_loss: 0.18925416
Epoch: [ 8/600] [   5/ 276] time: 4.7139s, d_loss: 1.51720512, g_loss: 0.79212797, rnn_loss: 0.25170082
Epoch: [ 8/600] [   6/ 276] time: 6.4613s, d_loss: 1.26051199, g_loss: 2.23050356, rnn_loss: 0.19912067
Epoch: [ 8/600] [   7/ 276] time: 4.6335s, d_loss: 1.33482277, g_loss: 1.52231884, rnn_loss: 0.24642673
Epoch: [ 8/600] [   8/ 276] time: 6.3295s, d_loss: 1.26564372, g_loss: 1.65149045, rnn_loss: 0.20058647
Epoch: [ 8/600] [   9/ 276] time: 5.8836s, d_loss: 1.39306271, g_loss: 2.62598825, rnn_loss: 0.27711484
Epoch: [ 8/600] [  10/ 276] time: 5.1058s, d_loss: 1.19326568, g_loss: 2.32397413, rnn_loss: 0.17292370
Epoch: [ 8/600] [  11/ 276] time: 5.6889s, d_loss: 1.20981526, g_loss: 3.77074933, rnn_loss: 0.22094548
Epoch: [ 8/600] [  12/ 276] time: 5.1492s, d_loss: 1.54772270, g

Epoch: [ 8/600] [  82/ 276] time: 4.1149s, d_loss: 1.00302458, g_loss: 2.72710228, rnn_loss: 0.22334883
Epoch: [ 8/600] [  83/ 276] time: 5.8480s, d_loss: 1.57552624, g_loss: 2.99288249, rnn_loss: 0.20993571
Epoch: [ 8/600] [  84/ 276] time: 3.6117s, d_loss: 1.53131151, g_loss: 0.88148350, rnn_loss: 0.14014331
Epoch: [ 8/600] [  85/ 276] time: 6.3284s, d_loss: 2.46832204, g_loss: 3.42512751, rnn_loss: 0.19344732
Epoch: [ 8/600] [  86/ 276] time: 4.0699s, d_loss: 1.28277636, g_loss: 2.32897329, rnn_loss: 0.19921796
Epoch: [ 8/600] [  87/ 276] time: 5.2142s, d_loss: 0.93257344, g_loss: 2.17476416, rnn_loss: 0.28360206
Epoch: [ 8/600] [  88/ 276] time: 4.2576s, d_loss: 1.22891593, g_loss: 2.61578107, rnn_loss: 0.22425175
Epoch: [ 8/600] [  89/ 276] time: 4.8258s, d_loss: 0.86697125, g_loss: 2.28077126, rnn_loss: 0.21091908
Epoch: [ 8/600] [  90/ 276] time: 4.3578s, d_loss: 1.03841519, g_loss: 1.56072235, rnn_loss: 0.27832195
Epoch: [ 8/600] [  91/ 276] time: 5.9290s, d_loss: 1.43139195, g

Epoch: [ 8/600] [ 161/ 276] time: 6.0676s, d_loss: 1.25713706, g_loss: 1.86090541, rnn_loss: 0.24698447
Epoch: [ 8/600] [ 162/ 276] time: 3.6856s, d_loss: 1.37408507, g_loss: 2.41895962, rnn_loss: 0.22080399
Epoch: [ 8/600] [ 163/ 276] time: 6.1081s, d_loss: 0.96049893, g_loss: 2.01659846, rnn_loss: 0.15570152
Epoch: [ 8/600] [ 164/ 276] time: 3.6275s, d_loss: 1.16055083, g_loss: 2.45525813, rnn_loss: 0.18615803
Epoch: [ 8/600] [ 165/ 276] time: 7.1927s, d_loss: 1.10683894, g_loss: 3.32752371, rnn_loss: 0.22002760
Epoch: [ 8/600] [ 166/ 276] time: 3.6850s, d_loss: 1.05875838, g_loss: 1.64037240, rnn_loss: 0.10370796
Epoch: [ 8/600] [ 167/ 276] time: 5.6224s, d_loss: 2.55429912, g_loss: 9.78082848, rnn_loss: 0.14731777
Epoch: [ 8/600] [ 168/ 276] time: 5.0474s, d_loss: 4.01158047, g_loss: 3.01826859, rnn_loss: 0.25886863
Epoch: [ 8/600] [ 169/ 276] time: 6.6169s, d_loss: 1.34412217, g_loss: 1.17286968, rnn_loss: 0.19496025
Epoch: [ 8/600] [ 170/ 276] time: 4.1417s, d_loss: 2.43384600, g

## 3. Evaluation metric

In [25]:
def generate_r_precision_data():
    caption_ids = np.reshape(np.asarray(test_dataset.captions_ids), (-1, cfg.TEXT.WORDS_NUM))
    captions_ids_wrong = np.reshape(test_dataset.random_wrong_captions(), (-1, cfg.WRONG_CAPTION, cfg.TEXT.WORDS_NUM))

    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    init = tf.global_variables_initializer()
    sess.run(init)

    # load the trained checkpoint
    checkpoint_dir = cfg.CHECKPOINT_DIR
    if checkpoint_dir is not None:
        loader = tf.train.Saver(var_list=tf.global_variables())
        ckpt_path = os.path.join(cfg.CHECKPOINT_DIR, cfg.CHECKPOINT_NAME)
        loader.restore(sess, ckpt_path)
        print("Restored model parameters from {}".format(ckpt_path))
    else:
        print('no checkpoints find.')

    n_caption_test = len(caption_ids)
    num_batches = n_caption_test // cfg.BATCH_SIZE

    true_cnn_features = np.zeros((num_batches, cfg.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM), dtype=float)
    true_rnn_features = np.zeros((num_batches, cfg.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM), dtype=float)
    wrong_rnn_features = np.zeros((num_batches, cfg.WRONG_CAPTION, cfg.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM), dtype=float)

    for i in range(num_batches):
        test_cap = caption_ids[i * cfg.BATCH_SIZE: (i + 1) * cfg.BATCH_SIZE]

        z = np.random.normal(loc=0.0, scale=1.0, size=(cfg.BATCH_SIZE, cfg.GAN.Z_DIM)).astype(np.float32)
        
        rnn_features = sess.run(rnn_encoder.outputs, feed_dict={t_real_caption: test_cap})
        gen = sess.run(generator.outputs, feed_dict={t_real_caption: test_cap, t_z: z})
        cnn_features = sess.run(cnn_encoder.outputs, feed_dict={t_real_image: gen})

        true_cnn_features[i] = cnn_features
        true_rnn_features[i] = rnn_features

        for per_wrong_caption in range(cfg.WRONG_CAPTION):
            test_cap = captions_ids_wrong[i * cfg.BATCH_SIZE: (i + 1) * cfg.BATCH_SIZE]
            rnn_features = sess.run(rnn_encoder.outputs, feed_dict={t_real_caption: test_cap[:, per_wrong_caption]})
            wrong_rnn_features[i, per_wrong_caption] = rnn_features
    
    # if exists, remove the existing file first
    try:
        os.remove(os.path.join(cfg.R_PRECISION_DIR, cfg.R_PRECISION_FILE))
    except OSError:
        pass
    np.savez(os.path.join(cfg.R_PRECISION_DIR, cfg.R_PRECISION_FILE), true_cnn=true_cnn_features, true_rnn=true_rnn_features,
             wrong_rnn=wrong_rnn_features)

In [27]:
def generate_inception_score_data():
    caption_ids = np.reshape(np.asarray(test_dataset.captions_ids),
                             (-1, cfg.TEXT.CAPTIONS_PER_IMAGE, cfg.TEXT.WORDS_NUM))
    
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    init = tf.global_variables_initializer()
    sess.run(init)

    checkpoint_dir = cfg.CHECKPOINT_DIR
    if checkpoint_dir is not None:
        loader = tf.train.Saver(var_list=tf.global_variables())
        ckpt_path = os.path.join(cfg.CHECKPOINT_DIR, cfg.CHECKPOINT_NAME)
        loader.restore(sess, ckpt_path)
        print("Restored model parameters from {}".format(ckpt_path))
    else:
        print('no checkpoints find.')

    n_caption_test = len(caption_ids)
    num_batches = n_caption_test // cfg.BATCH_SIZE

    for i in range(num_batches):
        for per_caption in range(cfg.TEXT.CAPTIONS_PER_IMAGE):
            test_cap = caption_ids[i * cfg.BATCH_SIZE: (i + 1) * cfg.BATCH_SIZE, per_caption]
            test_directory = test_dataset.filenames[i * cfg.BATCH_SIZE: (i + 1) * cfg.BATCH_SIZE]

            z = np.random.normal(loc=0.0, scale=1.0, size=(cfg.BATCH_SIZE, cfg.GAN.Z_DIM)).astype(np.float32)
            gen = sess.run(generator.outputs, feed_dict={t_real_caption: test_cap, t_z: z})
            
            for j in range(cfg.BATCH_SIZE):
                if not os.path.exists(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, test_directory[j].split('/')[0])):
                    os.mkdir(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, test_directory[j].split('/')[0]))

                scipy.misc.imsave(os.path.join(cfg.TEST.GENERATED_TEST_IMAGES, test_directory[j] + '_{}.png'.format(per_caption)), gen[j])

In [26]:
generate_r_precision_data()

INFO:tensorflow:Restoring parameters from ./checkpoint/model.ckpt
Restored model parameters from ./checkpoint/model.ckpt


In [28]:
generate_inception_score_data()

INFO:tensorflow:Restoring parameters from ./checkpoint/model.ckpt
Restored model parameters from ./checkpoint/model.ckpt


/home/duclv/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


## 3. Measure Inception score and R-precision of given test dataset

After set the config file as 'eval_birds.yml' and run the 'generate_inception_score_data()' and 'generate_r_precision_data()', the synthesized images based on given captions and set of image and caption features should be saved inside a 'evaluation' folder, specifically in 'evaluation/generated_images/..' and as 'evaluation/r_precision.npz' respectively.

**Then, go to the 'evaluation' folder and run each 'inception_score.ipynb' and 'r_precision.ipynb' file in order to measure inception score and r-precision score.**